In [ ]:
# Preparation for programming
# Make sure to execute this cell first!
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')                  # do not show warnings
from obspy import *
#from obspy import read
from obspy.core import read, UTCDateTime
from obspy.clients.fdsn import Client
import matplotlib.pylab as plt
import os
from os import path
import pandas as pd
from datetime import datetime 
import sys
import numpy as np
#from obspy.signal.invsim import cosine_taper

# 
# Applied Seismology, GEOS 626, University of Alaska Fairbanks
# Python coding by Nealey Sims
# Template script for analyzing modes spectra for sumatra
#
# Downloads all BH* and LH* data needed for the Sumatra B assignment
# 

# Plotting parameter
plt.rcParams['figure.figsize'] = 10, 5
plt.rcParams['lines.linewidth'] = 1

# path of the fft directory 
directoryfft = "./datawf/sumatra_fft"
# path to waveform directory
LHdirectory = "./datawf/sumatraLH"

sotime=UTCDateTime("2004-12-26T00:58:53.0")      # origin time of Sumatra earthquake)

In [ ]:
LHZst = read(LHdirectory+"/*Z",header=None)
all_amps = read(directoryfft+"/*amps", header = None)
all_f = np.load(directoryfft+'/all_fft_freq.npy',allow_pickle=True)
all_phs = np.load(directoryfft+'/all_fft_phase.npy',allow_pickle=True)
ind,ind_pdf,stas,chans,nets,tag,ikeep = np.genfromtxt(str(directoryfft)+'/sumatra_modes.txt', unpack=True, dtype=str)
ksta=[]
pickall=[]
w=LHZst.copy()
for k in range(len(stas)):
    if int(ikeep[k])==1:
        ksta.append(stas[k])
        pickall.append(int(ind_pdf[k]))

In [ ]:
client= Client("IRIS")
inven = client.get_stations(network="G,II,IU",starttime=sotime-(0.5*86400), endtime =sotime+(9*86400),  
                    channel="LHZ", level="response")

# USER: PICK A SET TO PLOT AND SAVE FOR ANALYSIS
# note: these are the same values as the 2nd column sumatra_modes.txt, use those values to index

#ipick = pickall;    # view all 133 not-cut stations
ipick=[0,1,2,19]
stkeep=[]
stan=[]
a_freqs=[]       
strmrs=[]
# remove instrument response
for i in ipick:
    for net in inven:
        for sta in net:
            for cha in sta: 
                if str(sta.code+cha.location_code) == str(ksta[i]):
                    # Get instrument response using the Inventory
                    Ia=cha.response.get_evalresp_response_for_frequencies(all_f[i],output='ACC')
    st_fft=all_amps[i].data
    C = st_fft*np.exp(1j*all_phs[i]) # Fourier Transform C(w)
    Xa = np.divide(C,Ia)             # remove instrument response
    stkeep.append(abs(C))
    strmrs.append(abs(Xa))
    stan.append(ksta[i])
    a_freqs.append(all_f[i])

In [ ]:
# Plot the full spectrum of the signal
for i in range(len(ipick)):
    indarray=stkeep[i]
    winind=[]
    for k,j in enumerate(a_freqs[i]):
        if (j >= .0002) and (j <= .001):
            winind.append(indarray[k])
    maxwin=max(winind)
    minwin=min(winind)
    plt.plot(a_freqs[i]*1000, abs(stkeep[i]), color='b')
    plt.title('Amplitude Spectrum of %7s (Zoomed in)' % (stan[i]))
    plt.xlabel('Frequency (mHz)')
    plt.ylabel('spectral amplitude (counts s)')
    plt.xlim(0.2, 1)
    plt.ylim(minwin, maxwin+2)
    #plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0))
    plt.show()
    
    plt.plot(a_freqs[i]*1000, abs(strmrs[i]), color='b')
    plt.title('-instrument response removed- Spectrum of %7s' % (stan[i]))
    plt.xlabel('Frequency (mHz)')
    plt.ylabel('spectral amplitude (acceleration) (m/s)')
    plt.xlim(0.2, 1)
    plt.ylim(0,7e-10)
    plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0))
    plt.show()
    

In [ ]:
# Example of stacking spectra
def w2fstack(freq,amp,f1,f2,n):
    # Function used to stack waveforms given frequency and amplitude arrays
    nw = len(freq)
    f = np.linspace(f1,f2,n)
    A = np.zeros((n,nw))
    for ii in range (nw):
        f0 = freq[ii]  # Hz
        A0 = amp[ii]
        A[:,ii] = np.interp(f,f0,A0)   
    Astack = np.sum(A,1)
    return Astack,f,A

f1 = (0.2*1E-3)
f2 = (10*1E-3)
n = int(1E5)

Astack,fplot,A = w2fstack(a_freqs,stkeep,f1,f2,n)
rAstack,rfplot,rA = w2fstack(a_freqs,strmrs,f1,f2,n)   # Stack stream with intrument response removed

plt.plot(fplot*1E3,Astack,'b')
plt.title('Stacked Spectrum with Instrument Response NOT removed')
plt.xlabel('frequency [mHz]')
plt.ylabel('stacked amplitude spectrum')
plt.show()

plt.plot(rfplot*1E3,rAstack,'b') 
plt.title('Stacked Spectrum with Instrument Response removed')
plt.xlabel('frequency [mHz]')
plt.ylabel('stacked amplitude spectrum')
plt.show()

# note: matrix A contains all spectra
plt.plot(rfplot,rA)
plt.xlabel('frequency [mHz]')
plt.ylabel('stacked amplitude spectrum')
plt.show()


In [ ]:
# START MODES PROBLEM HERE


#==========================================================================
